# Lesson 7.3: Integrating with External APIs and Data

---

In previous lessons, we built chatbots capable of conversation, search, and calculation. However, the true power of LLM applications lies in their ability to interact with the **external world** through **APIs (Application Programming Interfaces)** and databases. This lesson will focus on using **Agents** to connect LLMs with real-world web services and business systems, enabling chatbots to perform meaningful real-world tasks.

## 1. Using Agents to Interact with Real-World Web Services and Databases

When an LLM needs to perform tasks beyond its internal knowledge or the capabilities of general calculation/search tools, we need to integrate it with external systems. The **Agent** is the ideal component for this role.

* **Role of the Agent:** The Agent acts as an intelligent orchestrator. It receives user requests, infers intent, and decides whether a specific **Tool** needs to be called to complete the task. If so, it selects the appropriate Tool, formats the input, executes the Tool, and uses the result to generate the final response.
* **Benefits:**
    * **Extended Capabilities:** Allows the LLM to access up-to-date information, perform actions (e.g., create orders, send emails, schedule appointments), and interact with proprietary systems.
    * **Automate Business Tasks:** Transforms the chatbot into an assistant that can execute complex workflows.
    * **Reduce Hallucinations:** By retrieving information from authoritative sources, it reduces the LLM's tendency to fabricate data.




---

## 2. Building Tools to Call RESTful APIs

For the Agent to interact with external APIs, we need to build **Custom Tools**. Each Tool will encapsulate the logic to call a specific API.

* **`requests` library:** This is the standard Python library for making HTTP requests (GET, POST, PUT, DELETE) to RESTful APIs.
* **Tool Definition:**
    * Use the `@tool` decorator or LangChain's `Tool` class.
    * Provide a clear and unique `name`.
    * Write a detailed `description` that accurately describes the Tool's function, its required input parameters, and their format. The LLM will read this `description` to decide when and how to use the Tool.
    * Ensure the Tool's function returns a string that the LLM can read and understand.
* **Input/Output Handling:**
    * **Input:** The Tool can receive parameters from the LLM. You should use `Pydantic` to define the input schema, helping the LLM understand the necessary parameters.
    * **Output:** After calling the API, the Tool will process the API response (often JSON) and convert it into a human-readable text format for the LLM.


---

## 3. Real-World Examples (Conceptual)

Integrating with real-world APIs like Google Calendar, e-commerce databases, or CRM/ERP systems often requires complex steps such as:

* **Authentication:** OAuth 2.0, API keys, JWT tokens.
* **Client Libraries/SDKs:** API providers often offer Python libraries to simplify interaction.
* **State Management:** Maintaining sessions, access tokens.
* **Error Handling:** Catching specific API errors (401 Unauthorized, 404 Not Found, 500 Internal Server Error).

Due to this complexity and the requirements for API keys/environment configuration, we will cover the following examples conceptually and will use a **mock API** for the practical section.

* **Appointment Scheduling (Google Calendar API integration):**
    * **Tool:** `create_calendar_event(summary: str, start_time: str, end_time: str, attendees: List[str])`.
    * **Logic:** Uses the Google Calendar API Python client library, authenticates with OAuth, creates a new event.
    * **Response:** "Event 'Project Meeting' created for 10:00 on 2024-08-15."

* **Retrieving Product Information from an E-commerce Database:**
    * **Tool:** `get_product_details(product_id: str)`.
    * **Logic:** Connects to a database (e.g., PostgreSQL, MongoDB) or calls a RESTful API of the e-commerce system, queries product information by ID.
    * **Response:** "Product 'XYZ Phone' costs 15,000,000 VND, 100 units in stock."

* **Interacting with CRM/ERP Systems (e.g., Salesforce, SAP):**
    * **Tool:** `create_new_lead(name: str, email: str, company: str)`.
    * **Logic:** Uses the CRM/ERP system's SDK, authenticates, sends a request to create a new lead record.
    * **Response:** "New lead created for John Doe from ABC Corp."


---

## 4. Practical Example: Building an Agent that Can Perform Business Tasks via API

To illustrate API integration, we will build an Agent capable of managing orders through a **simple mock API**.

**Scenario:** An order management chatbot that can create new orders and check order status.

**Preparation:**
* Ensure you have `langchain-openai`, `pydantic` installed.
* Set the `OPENAI_API_KEY` environment variable.

In [ ]:
import os
from typing import Dict, Any, List, Optional
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from pydantic import BaseModel, Field
import time

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# --- 1. Mock Order API ---
# This is a mock in-memory order database
mock_orders_db = {}
order_id_counter = 0

def _generate_order_id():
    global order_id_counter
    order_id_counter += 1
    return f"ORDER-{order_id_counter:04d}"

def _mock_create_order_api(customer_name: str, product_name: str, quantity: int) -> Dict[str, Any]:
    """Simulates an API call to create a new order."""
    order_id = _generate_order_id()
    order_details = {
        "order_id": order_id,
        "customer_name": customer_name,
        "product_name": product_name,
        "quantity": quantity,
        "status": "Processing", # Initial status
        "timestamp": time.time() # For demonstration
    }
    mock_orders_db[order_id] = order_details
    return {"status": "success", "order_id": order_id, "details": order_details}

def _mock_get_order_status_api(order_id: str) -> Dict[str, Any]:
    """Simulates an API call to get order status."""
    order = mock_orders_db.get(order_id)
    if order:
        return {"status": "success", "order_id": order_id, "details": order}
    else:
        return {"status": "error", "message": f"Order with ID: {order_id} not found"}

print("Mock order API initialized.")

# --- 2. Define Custom Tools to interact with the mock API ---

# Input Schema for Create Order Tool
class CreateOrderInput(BaseModel):
    customer_name: str = Field(description="Name of the customer placing the order.")
    product_name: str = Field(description="Name of the product being ordered.")
    quantity: int = Field(description="Quantity of the product being ordered.")

@tool("create_new_order", args_schema=CreateOrderInput)
def create_new_order_tool(customer_name: str, product_name: str, quantity: int) -> str:
    """
    Creates a new order with customer name, product name, and quantity.
    Returns the order ID and status.
    """
    if quantity <= 0:
        return "Error: Product quantity must be greater than 0."
    
    response = _mock_create_order_api(customer_name, product_name, quantity)
    if response["status"] == "success":
        return f"Order created successfully! Your order ID is {response['order_id']}. Status: {response['details']['status']}."
    else:
        return f"Error creating order: {response['message']}"

# Input Schema for Get Order Status Tool
class GetOrderStatusInput(BaseModel):
    order_id: str = Field(description="The unique ID of the order to check status for.")

@tool("get_order_status", args_schema=GetOrderStatusInput)
def get_order_status_tool(order_id: str) -> str:
    """
    Retrieves the current status of an order based on the order ID.
    """
    response = _mock_get_order_status_api(order_id)
    if response["status"] == "success":
        details = response["details"]
        return (f"Order {details['order_id']} for customer {details['customer_name']} "
                f"for product '{details['product_name']}' (quantity {details['quantity']}) is: {details['status']}.")
    else:
        return f"Error getting order status: {response['message']}"

tools = [create_new_order_tool, get_order_status_tool]
print("Custom Tools defined.")

# --- 3. Initialize LLM and Agent ---
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Define Prompt for Agent
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an order management assistant. You have access to the following tools: {tools}. Use them to create orders or check order status. Respond politely and helpfully."),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("human", "{input}"),
])

# Create Agent
agent = create_react_agent(llm, tools, agent_prompt)

# Create Agent Executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True, # Enable verbose to see Agent's thought process
    handle_parsing_errors=True # Allow Agent to attempt self-correction
)

print("Agent and Agent Executor initialized.")

# --- 4. Execute and Test Agent ---
print("\n--- Starting Order Management Agent Test ---")

# Scenario 1: Create a new order
query_1 = "I want to order 2 mechanical keyboards for Nguyen Van A."
print(f"\nUser: {query_1}")
response_1 = agent_executor.invoke({"input": query_1})
print(f"Agent: {response_1['output']}")

# Scenario 2: Check order status (using ID from the newly created order)
# Get order ID from mock_orders_db (or from previous response if you want to automate)
first_order_id = list(mock_orders_db.keys())[0] if mock_orders_db else "ORDER-0001"
query_2 = f"Check the status of order {first_order_id}."
print(f"\nUser: {query_2}")
response_2 = agent_executor.invoke({"input": query_2})
print(f"Agent: {response_2['output']}")

# Scenario 3: Check non-existent order status
query_3 = "Check the status of order ORDER-9999."
print(f"\nUser: {query_3}")
response_3 = agent_executor.invoke({"input": query_3})
print(f"Agent: {response_3['output']}")

# Scenario 4: Request unrelated to Tool functionality
query_4 = "What's the weather like today?"
print(f"\nUser: {query_4}")
response_4 = agent_executor.invoke({"input": query_4})
print(f"Agent: {response_4['output']}")

print("\n--- Order Management Agent Test Ended ---")